# Bootstrap Your Own Latent (BYOL)

In this session we are going to implement Bootstrap Your Own Latent paper (https://arxiv.org/abs/2006.07733).

It uses a MoCo-style training (with asymmetric SiameseNet) but with a L2 loss penalty (it is not a contrastive-base method).

L'implementazione BYOL è un po' più complessa degli altri metodi. 

BYOL si basa su MoCo ma quest'ultimo usa la memoria per salvarsi le precedenti positive per poi usarle come negative. I BYOL la memoria non serve più, perché non è basato su Contrastive Loss, quindi non abbiamo più proprio il concetto di positive e negative, quindi non serve nemmeno più la memoria. Usa una distanza L2

BYOL però da MoCo prende la struttura asimmetrica della siamese network. Ha una target network che è la copia dello student network, ma che viene aggiornata in modo particolare. Questo aggiornamento della target network è chiamato EMA update.

BYOL deve gestire questa asimmetria delle reti e poi anche questo diverso aggiornamento dei pesi: lo student viene aggiornato con SGD normalmente (o con un ottimizzatore normale), mentre la teacher viene aggiornato con una tecnica di momentum.

barlow twins vs byol: memoria, byol non è basato su contrastive loss, non serve neppure più la memoria, utilizza una loss l2 (distanza euclidea tra features, tra due versioni augmented della stessa immagine). riprende la struttura asimmetrica della siamese network. l'aggiornamento della target net si chiama ema.

qui non servirà la siamese network

In [ ]:
import os
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torchvision.io import read_image

import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

from PIL import Image
import copy

In [ ]:
# loss fn
def loss_fn(x, y): # normalizza le features
    x = F.normalize(x, dim=-1, p=2) # p è l'esponente
    y = F.normalize(y, dim=-1, p=2)
    return 2 - 2 * (x * y).sum(dim=-1) # quadrato di un binomio, meno 2 volte il prodotto tra loro due


class EMA():
    # exponential moving average
    def __init__(self, beta):
        super().__init__()
        self.beta = beta

    def update_average(self, old, new): # i parametri vecchi e parametri nuovi
        if old is None:
            return new
        return old * self.beta + (1 - self.beta) * new # classico momentum in codice

def update_moving_average(ema_updater, ma_model, current_model):
    for current_params, ma_params in zip(current_model.parameters(), ma_model.parameters()): # zip prende un parametro da una lista e poi dall'altra, vuole due iteratori
        old_weight, up_weight = ma_params.data, current_params.data # si aggiorna la rete così per ogni parametro della rete
        ma_params.data = ema_updater.update_average(old_weight, up_weight)

# MLP class for projector and predictor

def MLP(dim, projection_size, hidden_size=4096, sync_batchnorm=None):
    return nn.Sequential(
        nn.Linear(dim, hidden_size),
        nn.BatchNorm1d(hidden_size),
        nn.ReLU(inplace=True),
        nn.Linear(hidden_size, projection_size)
    )


class BYOL(nn.Module): # non è più una loss, ma bensì ha dentro anche la parte di rete. L'uscità non saranno più le features, ma direttamente la loss
    def __init__(self, backbone, moving_average_decay = 0.99): # prende la backbone (siamese) e basta, il mad è il lambda con valore di default
        super().__init__()

        self.target_ema_updater = EMA(moving_average_decay)

        self.online_net = backbone # la backbone viene usata per definire la online net
        self.online_net.fc = nn.Identity() # togliamo fc e mettiamo la identity, il fc non viene sovrascritto (immagine rete) il project ce lo h solo la online network
        self.online_projector = MLP(512, 512, 4096) # il project è un mlp, solo l'online network ha un projector aggiuntivo

        self.target_net = None # qui non lo consideriamo il projector

    def _get_target_encoder(self): # prende la target net come copia della online net
        if self.target_net is None:
          # init target net
          target_net = copy.deepcopy(self.online_net)
          for p in target_net.parameters():
              p.requires_grad = False
          self.target_net = target_net
        else:
          target_net = self.target_net
        return target_net

    def update_moving_average(self):
        update_moving_average(self.target_ema_updater, self.target_net, self.online_net) # prende la rete allo stato attuale..

    def forward(self, x1, x2): # qui non arrivano le features, ma direttamente le immagini

        images = torch.cat((x1, x2), dim = 0) # concatenazione versione augmented

        online_projections = self.online_projector(self.online_net(images)) # q_theta di zeta_theta, prendo le features delle encoder e poi ci si appende un predictor in più
        online_pred_one, online_pred_two = online_projections.chunk(2, dim = 0) # ri-splitto le projection sulle righe, chunck divide un vettore in toto (2) dimensioni

        with torch.no_grad(): # di tutto quello che c'è qui dentro a noi non interessa calcolarci i gradienti, non vogliamo farci la backprop
            target_net = self._get_target_encoder() # si prende il target encoder # ogni volta copia la online net, ma non va bene ( _get_target_encoder!! (corretta)

            target_projections = target_net(images) # si fa forward della rete
            target_projections = target_projections.detach()
            target_proj_one, target_proj_two = target_projections.chunk(2, dim = 0) # rifacciamo i chunk

            # perché (immagine) si fa il forward delle due versioni augmented? la versione online ne processa una (x1), la target un'altra (x2), ma perché è comodo farlo con entrambe le reti?
            # (anche o vede x2 e t x2)
            # perché così è più efficiente! altrimenti sarebbe l1 (vedi disegno)
            # ecco perché vengono calcolate due loss e poi sommate, di cui poi si prende la media

        loss_one = loss_fn(online_pred_one, target_proj_two.detach())
        loss_two = loss_fn(online_pred_two, target_proj_one.detach())

        loss = loss_one + loss_two
        return loss.mean()

## Exercise 0

Study the above code.
- Where is the EMA updates?
- Why it computes both loss_one and loss_two values?

## Exercise 1

Write the training loop for moco-style training as used in BYOL.
Use the Dataset which creates the two augmented views for each image and the Siamese Network from the past lab session [1](https://colab.research.google.com/drive/1NJwAFbRiD4MdwWf__6P2Lm0xYk_DNdVu?usp=sharing) and [2](https://colab.research.google.com/drive/1AMkh0q8L5nJScx7v6cMWoK336zqOqDY6?usp=sharing).

In [ ]:
backbone = models.resnet18()
#print(backbone)

#! remember to delete the fc layer (we need just the CNN layers + flatten)
backbone.fc = nn.Identity()

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, data, targets = None, transform=None, target_transform=None): # valori di default
        self.imgs = data
        self.targets = targets
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img = self.imgs[idx]
        if isinstance(img, str):
          image = read_image(img)
        else:
          image = Image.fromarray(img.astype('uint8'), 'RGB')
        if self.transform: # arriva qui con una PIL image
            image1 = self.transform(image) # fa due trasformazioni
            image2 = self.transform(image)
        else:
            image1 = image
            image2 = image
        return image1, image2

In [ ]:
class Identity(torch.nn.Module):
  def forward(self, x):
    return x

In [ ]:
data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True) # dataset

color_jitter = transforms.ColorJitter(0.8, 0.8, 0.8, 0.2) # si distorce il colore con una certa probabilità tutti i canali


transform = transforms.Compose([transforms.RandomResizedCrop(size=32),
                                  transforms.RandomHorizontalFlip(),
                                  transforms.RandomApply([color_jitter], p=0.8),
                                  transforms.RandomGrayscale(p=0.2),
                                  transforms.GaussianBlur(kernel_size=int(0.1 * 32)),
                                  transforms.ToTensor()])

trainset = CustomImageDataset(data.data, transform = transform)
dataloader = DataLoader(trainset, batch_size=64, shuffle=True)


byol = BYOL(models.resnet18())

online_net_params = {'params': byol.online_net.parameters()}
online_project_params = {'params': byol.online_projector.parameters()}

optimizer = torch.optim.SGD([online_net_params, online_project_params], lr = 0.4, momentum = 0.9, weight_decay=1e-04)

# qui c'è un doppio aggiornamento: aggiornamento SGD e aggiornamento EMA
# EMA deve andare dopo SGD: prima si aggiorna il modello student e poi il target. Chiaramente il target deve aggiornarsi anche sullo stato attuale dello student

for idx, (v1, v2) in enumerate(dataloader):
    optimizer.zero_grad()
    loss = byol(v1, v2) # gli diamo in ingresso le due view
    loss.backward()

    optimizer.step() # sgd for online net
    byol.update_moving_average() # ema for target net


    print(f"batch {idx} loss {loss.item()}")
    print()

    if idx == 3:
        break

Files already downloaded and verified
batch 0 loss 4.0882887840271

batch 1 loss 1.4373500347137451

batch 2 loss 1.4093679189682007

batch 3 loss 1.4153363704681396

